In [1]:
!pip install wandb -qU
!pip install keras-self-attention

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=00122dc7acbc925ad65ccd2b6e5c83162233ff785969d01540e29bc13e735823
  Stored in directory: /root/.cache/pip/wheels/b8/f7/24/607b483144fb9c47b4ba2c5fba6b68e54aeee2d5bf6c05302e
Successfully built keras-self-attention


In [2]:
from keras.layers import Input, Dense, Flatten, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn import metrics
from sklearn.model_selection import train_test_split

from google.colab import drive

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os   # This module is used for interacting with the operating system. It provides a way to work with files and directories.

import tensorflow as tf
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

In [3]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# --- Load data and setup directories ---
drive.mount('/content/gdrive')
!unzip gdrive/MyDrive/TCGA-BRCA_1079.zip

#'r' before the string indicates a raw string ('\' are treated as literal and not escape characters)
result_path = r"results"

if not os.path.exists(result_path):
    os.makedirs(result_path)

Mounted at /content/gdrive
Archive:  gdrive/MyDrive/TCGA-BRCA_1079.zip
  inflating: TCGA-BRCA_1079.Xena_TCGA_PanCan.annotation_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv  
  inflating: TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv  


In [5]:
class AE:
    def __init__(self,data,dims):
        self.data = data
        self.x, self.x_val = train_test_split(self.data, test_size=0.1, train_size=0.9)

        self.input_dim = data.shape[1]
        self.encoding1_dim = dims[0]
        self.encoding2_dim = dims[1]
        self.encoding3_dim = dims[2]
        self.middle_dim = dims[3]

        self.input_factor = Input(shape=(self.input_dim,))
        self.encoded = Dense(self.encoding1_dim, activation='sigmoid')(self.input_factor)
        self.encoded = Dense(self.encoding2_dim, activation='sigmoid')(self.encoded)
        self.encoded = Dense(self.encoding3_dim, activation='sigmoid')(self.encoded)
        self.encoded = Reshape((self.encoding3_dim, 1))(self.encoded) # Self-attention layer input needs to be a sequence
        self.encoded = SeqSelfAttention(attention_activation='linear')(self.encoded)
        self.encoded = Flatten()(self.encoded)

        self.encoder_output = Dense(self.middle_dim, activation='sigmoid')(self.encoded)

        self.decoded = Dense(self.encoding3_dim, activation='sigmoid')(self.encoder_output)
        self.decoded = Reshape((self.encoding3_dim, 1))(self.decoded)
        self.decoded = SeqSelfAttention(attention_activation='linear')(self.decoded)
        self.decoded = Flatten()(self.decoded)
        self.decoded = Dense(self.encoding2_dim, activation='sigmoid')(self.decoded)
        self.decoded = Dense(self.encoding1_dim, activation='sigmoid')(self.decoded)
        self.decoded = Dense(self.input_dim)(self.decoded)

        self.autoencoder = Model(inputs=self.input_factor, outputs=self.decoded)

        self.encoder = Model(inputs=self.input_factor, outputs=self.encoder_output)

        # compile autoencoder
        # the initial learning rate for the 'adam' optimizer is 0.001
        custom_adam_optimizer = Adam(learning_rate=0.0005)
        self.autoencoder.compile(optimizer=custom_adam_optimizer, loss='mse')

    def train(self):
        # This callback will log training and validation metrics along with system metrics to W&B
        callback_metrics = WandbMetricsLogger()
        early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=50, min_delta=0.01, restore_best_weights=True)
        self.autoencoder.fit(self.x, self.x, validation_data=(self.x_val, self.x_val), epochs=500, verbose=1, batch_size=32, shuffle=True, callbacks=[callback_metrics, early_stopping])

    def save(self):
        self.autoencoder.save("./trained_model/model.h5")

    def predict(self,data):
        return self.encoder.predict(data)

In [6]:
# --- Prepare input data ---
input_data_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.log2_exprs_z_v6.tsv", sep="\t")   # read .tsv file into memory
input_data = input_data_frame.values  # retrieve values as numpy array
print ("Shape of input data: ", input_data.shape) # rows are genes and columns are samples
print(input_data)

input_data = np.transpose(input_data) # rows are samples and columns are genes
input_data = np.delete(input_data, (0), axis=0) # delete first row of the data matrix
print ("Shape of prepared input data: ", input_data.shape)
print(input_data)

input_data = np.asarray(input_data).astype("float32") # Every character/number encoded with 32 Bit
print("--- Converted to Float32 ---")
print(input_data)

Shape of input data:  (17162, 1080)
[['A1BG' 0.1719444908272692 0.7300083449184781 ... 0.614468050540426
  1.3458846686508648 0.7186659277414379]
 ['A1CF' -0.814924951368837 2.386837959210663 ... 3.016626468680079
  -0.814924951368837 0.2356343444931301]
 ['A2M' -0.946474342215696 -0.5321448682087864 ... 0.5601782905367024
  1.512226893819039 -0.6446442624050204]
 ...
 ['DHCR7' 0.9104445032812696 0.3842357893348481 ... 0.7718469491250562
  -0.4217551872682497 -0.970509103357415]
 ['TMEM45B' -1.9635765960806173 1.268061022702539 ... -0.3477150909879061
  0.4043269887748967 -0.5444141861092432]
 ['GPR160' 0.6427457224400778 1.759067645972089 ... 0.3169998035848742
  -0.7956005603903231 -0.1338926768376846]]
Shape of prepared input data:  (1079, 17162)
[[0.1719444908272692 -0.814924951368837 -0.946474342215696 ...
  0.9104445032812696 -1.9635765960806173 0.6427457224400778]
 [0.7300083449184781 2.386837959210663 -0.5321448682087864 ...
  0.3842357893348481 1.268061022702539 1.759067645972

In [7]:
# --- Prepare labels ---
subtype_labels_frame = pd.read_csv("TCGA-BRCA_1079_17Kgenes.Xena_TCGA_PanCan.subtypes_and_signatures_v6.tsv", sep="\t")
subtype_labels = subtype_labels_frame.values
print ("Shape of subtype labels: ", subtype_labels.shape)
print(subtype_labels)

subtype_label_set = set(subtype_labels[:,1])
print(subtype_label_set)

Shape of subtype labels:  (1079, 34)
[['TCGA-3C-AAAU-01' 'LumB' 0 ... 2 1 0]
 ['TCGA-3C-AALI-01' 'Her2' 0 ... 3 0 0]
 ['TCGA-3C-AALJ-01' 'LumB' 0 ... 3 0 0]
 ...
 ['TCGA-XX-A89A-01' 'LumA' 0 ... 1 0 0]
 ['TCGA-Z7-A8R5-01' 'LumA' 0 ... 1 0 0]
 ['TCGA-Z7-A8R6-01' 'LumB' 0 ... 3 0 0]]
{'Normal', 'LumA', 'Her2', 'LumB', 'Basal'}


In [8]:
encoding1_dim = 8000
encoding2_dim = 1000
encoding3_dim = 200
middle_dim = 10
dims = [encoding1_dim, encoding2_dim, encoding3_dim, middle_dim]


for i in range(3, 11):
    # initialize a new run (<=> single unit of computation)
    run = wandb.init(project="Attention_AE_fitting",
                     # Set the project where this run will be logged
                     name=f"early_stopping_patience_50",  # We pass a run name otherwise it’ll be randomly assigned
                     # Track hyperparameters and run metadata
                     config={
                         "optimizer": "adam",
                         "loss": "mse",
                         "training_set": "BRCA",
                         "validation split": 0.1,
                         "test_set": "none",
                         "batch_size": 32,
                         "encoding_layers": 3,
                         "activation_functions": "sigmoid, sigmoid, sigmoid, identity",
                         "self-attention layers": "1 in encoder & decoder"
                     })

    ae = AE(input_data, dims)
    ae.autoencoder.summary()
    ae.train()

    # Save embedded encodings
    encoded_factors = ae.predict(input_data)
    if not os.path.exists("{p}/Attention_AE_EM_{i}.txt".format(p=result_path, i=i)):
        os.mknod("{p}/Attention_AE_EM_{i}.txt".format(p=result_path, i=i))
    np.savetxt("{p}/Attention_AE_EM_{i}.txt".format(p=result_path, i=i), encoded_factors)

    # Mark the run as finished
    wandb.finish()

wandb: Currently logged in as: cosybio-compsysmed. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17162)]           0         
                                                                 
 dense (Dense)               (None, 8000)              137304000 
                                                                 
 dense_1 (Dense)             (None, 1000)              8001000   
                                                                 
 dense_2 (Dense)             (None, 200)               200200    
                                                                 
 reshape (Reshape)           (None, 200, 1)            0         
                                                                 
 seq_self_attention (SeqSel  (None, 200, 1)            129       
 fAttention)                                                     
                                                             

31/31 [==============================] - 10s 92ms/step - loss: 1.1147 - val_loss: 1.0352
Epoch 2/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9958 - val_loss: 1.0339
Epoch 3/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9955 - val_loss: 1.0339
Epoch 4/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9954 - val_loss: 1.0339
Epoch 5/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9954 - val_loss: 1.0340
Epoch 6/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9954 - val_loss: 1.0341
Epoch 7/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9953 - val_loss: 1.0341
Epoch 8/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9953 - val_loss: 1.0342
Epoch 9/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9953 - val_loss: 1.0343
Epoch 10/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9953 - val_loss: 1.0343
Epoch 11/5

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▄▄▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,████████▇▇▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,121
epoch/learning_rate,0.0005
epoch/loss,0.89856
epoch/val_loss,0.95682


...

31/31 [==============================] - 7s 78ms/step - loss: 1.1177 - val_loss: 1.0183
Epoch 2/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9978 - val_loss: 1.0170
Epoch 3/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9974 - val_loss: 1.0170
Epoch 4/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9973 - val_loss: 1.0169
Epoch 5/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9973 - val_loss: 1.0170
Epoch 6/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9972 - val_loss: 1.0171
Epoch 7/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9972 - val_loss: 1.0171
Epoch 8/500
31/31 [==============================] - 1s 24ms/step - loss: 0.9972 - val_loss: 1.0171
Epoch 9/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9972 - val_loss: 1.0172
Epoch 10/500
31/31 [==============================] - 1s 25ms/step - loss: 0.9971 - val_loss: 1.0172
Epoch 11/50

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██████▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,███████▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,181
epoch/learning_rate,0.0005
epoch/loss,0.89839
epoch/val_loss,0.9112


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 17162)]           0         
                                                                 
 dense_56 (Dense)            (None, 8000)              137304000 
                                                                 
 dense_57 (Dense)            (None, 1000)              8001000   
                                                                 
 dense_58 (Dense)            (None, 200)               200200    
                                                                 
 reshape_14 (Reshape)        (None, 200, 1)            0         
                                                                 
 seq_self_attention_14 (Seq  (None, 200, 1)            129       
 SelfAttention)                                                  
                                                          

31/31 [==============================] - 6s 79ms/step - loss: 1.1277 - val_loss: 0.9758
Epoch 2/500
31/31 [==============================] - 1s 25ms/step - loss: 1.0023 - val_loss: 0.9749
Epoch 3/500
31/31 [==============================] - 1s 24ms/step - loss: 1.0020 - val_loss: 0.9749
Epoch 4/500
31/31 [==============================] - 1s 24ms/step - loss: 1.0020 - val_loss: 0.9750
Epoch 5/500
31/31 [==============================] - 1s 24ms/step - loss: 1.0019 - val_loss: 0.9752
Epoch 6/500
31/31 [==============================] - 1s 24ms/step - loss: 1.0019 - val_loss: 0.9752
Epoch 7/500
31/31 [==============================] - 1s 24ms/step - loss: 1.0018 - val_loss: 0.9753
Epoch 8/500
31/31 [==============================] - 1s 25ms/step - loss: 1.0017 - val_loss: 0.9754
Epoch 9/500
31/31 [==============================] - 1s 25ms/step - loss: 1.0017 - val_loss: 0.9755
Epoch 10/500
31/31 [==============================] - 1s 25ms/step - loss: 1.0016 - val_loss: 0.9756
Epoch 11/50

epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▄▄▄▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,████████▇▇▇▆▅▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
epoch/epoch,124
epoch/learning_rate,0.0005
epoch/loss,0.89817
epoch/val_loss,0.90944
